In [134]:
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [135]:
scene_name = "bicycle"
local = False
alt = ""
local_text = "_local" if local else ""

scenes = ["berlin", "nyc", "london_v5", "alameda_v11"]
#scenes = ["bicycle", "bonsai", "counter", "garden", "stump"]

metrics_to_process = ["psnr", "ssim", "lpips"]
best_is_max = {"psnr": True, "ssim": True, "lpips": False}

In [136]:
baseline_dict = {
    "EVER": {
        "berlin": [27.24, 0.900, 0.371],
        "nyc": [27.93, 0.863, 0.337],
"alameda": [24.72, 0.779, 0.389],
"london": [26.49, 0.837, 0.374],

    "bicycle": [25.34, 0.776, 0.220],
        "garden": [27.46, 0.869, 0.120],
        "stump": [26.41, 0.781, 0.230],
        "counter": [28.91, 0.910, 0.24],
        "bonsai": [32.24, 0.943, 0.236]
    },
    "GSplat-7k": {
        "bicycle": [23.71, 0.662, 0.324],
        "garden": [26.30, 0.833, 0.123],
        "stump": [25.62, 0.720, 0.253],
        "counter": [27.14, 0.878, 0.206],
        "bonsai": [29.66, 0.922, 0.162]
    },
    "3DGS MCMC": {
        "counter": [29.51, 0.92, 0.22],
        "stump": [27.80, 0.82, 0.19],
        "bicycle": [26.15, 0.81, 0.18],
        "bonsai": [32.88, 0.95, 0.22],
        "garden": [28.16, 0.89, 0.10]
    }
}

In [137]:
global_training_times = {
    "bicycle": {
    100: 3.0,
    500: 14.2,
    1000: 27.9,
    2000: 55.6,
    10000: 259.5
},
    "counter": {
    100: 5.6,
    500: 16.9,
    1000: 30.9,
    2000: 59.3,
    10000: 268.0
},
    "berlin": {
    1000: 30.1,
    2000: 58.1,
    5000: 134.7,
    10000: 260.3
},
    "nyc": {
    1000: 31.49,
    2000: 59.35,
    5000: 135.27,
    10000: 261.06
},
    "london": {
    1000: 35.34
},
    "alameda": {
    1000: 36.97
},
    "bonsai": {
    100: 6.86,
    500: 17.71,
    1000: 32.13,
    2000: 59.68,
    10000: 264.24
},
    "garden": {
    100: 4.19,
    500: 15.36,
    1000: 29.06,
    2000: 56.59,
    10000: 258.29
},
    "stump": {
    100: 3.36,
    500: 14.24,
    1000: 28.19,
    2000: 56.14,
    10000: 259.71
}
}
local_training_times_by_scene = {
    "berlin": {
    200: 5.78 + 5.57 + 5.68 + 5.58 + 5.63,
    1000: 27.79 + 27.87 + 27.89 + 27.87 + 27.69,
    2000: 55.60 + 55.16 + 56.08 + 56.00 + 55.47,
    5000: 132.34 + 131.59 + 130.79 + 132.46 + 131.65,
    10000: 255.00 + 254.43 + 253.70 + 254.41 + 254.09,
    "1000_merged": 29.09
},
    "nyc": {
    200: 5.89 + 5.67 + 6.00 + 5.83 + 6.06,
    1000: 28.24 + 27.60 + 27.89 + 27.91 + 27.26,
    2000: 57.28 + 55.26 + 55.74 + 54.85 + 55.43,
    "1000_merged": 30.11
},
    "london": {
    "1000_merged": 30.14
},
    "alameda": {
        "1000_merged": 30.30
    }
}

In [138]:
training_times_by_global_nerf_iterations = None
training_times_by_local_nerf_iterations = None
training_times = dict()


In [139]:
def compute_scene_results(scene, baseline_experiment="SfM"):
    data_by_metric = {}
    name_map = {}
    training_times = {}

    for metric in metrics_to_process:
        log_dir = f"../tensorboard_results/{scene}{alt}{local_text}/{metric}"
        dfs = []
        for root, dirs, files in os.walk(log_dir):
            for file in files:
                if file.endswith(".csv"):
                    # For berlin, skip files that contain "100k"
                    if "processed_splatfacto_sparse_pc_30000_its" not in file:
                        if scene != "stump":
                            if scene == "berlin" and "100k" in file:
                                continue
                            #elif scene == "bicycle" and (("1000" not in file) or "sparse" in file):
                            #    continue
                            #elif ("sparse_pc" not in file) and ((("150000" not in file) and ("150k" not in file))):
                            #    continue
                            #elif ("200" in file) or ("500its" in file) or ("100its" in file) or ("10000its" in file) or ("add" in file):
                            #    continue
                        elif "nerf_10000iters_init_30000_its" in file or "nerf_1000iters_init_30000_its" in file:
                            continue
                    full_path = os.path.join(root, file)
                    df = pd.read_csv(full_path)

                    experiment_name = os.path.splitext(file)[0]
                    experiment_name = experiment_name.split("splatfacto_")[-1]
                    experiment_name = (experiment_name.replace("1mil", "1000000")
                                                   .replace("1m", "1000000")
                                                   .replace("1k", "1000")
                                                   .replace("0k", "0000")
                                                   .replace("add_", ""))
                    df['experiment'] = experiment_name
                    dfs.append(df)

                    # Build the mapping and record training time if not already set.
                    if experiment_name not in name_map:
                        if ("merged_sfm" in experiment_name) or ("sfm_global" in experiment_name):
                            pattern = r'(\d+)its_(\d+)pts_(\d+)_its'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                points = int(match.group(2))
                                display_name = f"Global NeRF ({nerf_iterations}its) + SfM"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                pattern = r'(\d+)its_(\d+)_'
                                match = re.search(pattern, experiment_name)
                                if match:
                                    nerf_iterations = int(match.group(1))
                                    display_name = f"Global NeRF ({nerf_iterations}its) + SfM"
                                    name_map[experiment_name] = display_name
                                    training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                                else:
                                    name_map[experiment_name] = experiment_name
                        elif "sparse_pc" in experiment_name:
                            name_map[experiment_name] = "SfM"
                        elif (("local_merged" in experiment_name) or ("merged_local" in experiment_name)) and ("1000" in experiment_name):
                            name_map[experiment_name] = "Local NeRFs+ SfM"
                            training_times[name_map[experiment_name]] = local_training_times_by_scene.get(scene.split("_")[0], {}).get("1000_merged", 0.0)
                        elif "local_nerf" in experiment_name:
                            pattern = r'(\d+)its_(\d+)'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                points = int(match.group(2))
                                display_name = f"Local NeRFs ({nerf_iterations}its, {points} pts) + SfM"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = local_training_times_by_scene.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                name_map[experiment_name] = experiment_name
                        elif "global_nerf" in experiment_name:
                            pattern = r'(\d+)its_(\d+)_'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                points = int(match.group(2))
                                display_name = f"Global NeRF ({nerf_iterations}its, {points} pts)"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                name_map[experiment_name] = experiment_name
                        elif "nerf" in experiment_name:
                            pattern = r'(\d+)_iters'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                display_name = f"Global NeRF ({nerf_iterations}its) only"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                name_map[experiment_name] = experiment_name
                        else:
                            print("DROPPING", experiment_name)
                            continue
                            name_map[experiment_name] = experiment_name
        if dfs:
            data_by_metric[metric] = pd.concat(dfs, ignore_index=True)

    def compute_best_within_baseline(df, experiments, baseline_experiment, best_is_max, by="time"):
        # Use only experiments that are in our mapping.
        df = df[df['experiment'].isin(experiments.keys())].copy()
        df['experiment_display'] = df['experiment'].map(experiments)
        if "Time" not in df.columns and "Wall time" in df.columns:
            df = df.rename(columns={"Wall time": "Time"})
        x = "Time" if by=="time" else "Step"
        baseline_df = df[df['experiment_display'].str.lower() == baseline_experiment.lower()]
        if baseline_df.empty:
            baseline_duration = float('inf')
        else:
            baseline_duration = baseline_df[x].max() - baseline_df[x].min()
        best_dict = {}
        for exp, group in df.groupby('experiment_display'):
            if by=="time":
                startup = training_times.get(exp, 0.0)
                if startup == 0.0 and exp != baseline_experiment:
                    print(f"Warning: no training time found for {exp}, {scene_name}")
                group[x] = group[x] - group[x].min() + startup
            group_within = group[group[x] <= baseline_duration]
            if group_within.empty:
                best_val = None
            else:
                best_val = group_within['Value'].max() if best_is_max else group_within['Value'].min()
            best_dict[exp] = best_val
        return best_dict

    # Compute best values for each metric.
    best_results = {}
    for metric in metrics_to_process:
        if metric not in data_by_metric:
            continue
        best_vals = compute_best_within_baseline(
            data_by_metric[metric],
            name_map,
            baseline_experiment,
            best_is_max[metric],
            by="time"
        )
        best_results[metric] = best_vals

    # Merge best values into a single results dictionary.
    scene_results = {}
    for metric in metrics_to_process:
        if metric in best_results:
            for exp, val in best_results[metric].items():
                if exp not in scene_results:
                    scene_results[exp] = {}
                col_name = f"Best {metric.upper()}"
                scene_results[exp][col_name] = val

    results_df = pd.DataFrame.from_dict(scene_results, orient='index').reset_index().rename(columns={"index": "Experiment"})
    return results_df

results_per_scene = {}
for scene in scenes:
    df_scene = compute_scene_results(scene, baseline_experiment="SfM")
    # Use Experiment as index, then rename metric columns to include scene name.
    df_scene = df_scene.set_index("Experiment")
    rename_dict = {}
    for col in df_scene.columns:
        if col.startswith("Best "):
            metric = col.split(" ")[1]
            rename_dict[col] = f"{scene}_{metric}"
    df_scene = df_scene.rename(columns=rename_dict)
    results_per_scene[scene] = df_scene

merged_results = None
for scene, df in results_per_scene.items():
    if merged_results is None:
        merged_results = df.copy()
    else:
        merged_results = merged_results.join(df, how="outer")
merged_results = merged_results.reset_index()

final_results = merged_results.copy()
# Only keep experiments that appear in all scenes

baseline_rows = []
for baseline_name, scene_values in baseline_dict.items():
    row = {"Experiment": baseline_name}
    for col in final_results.columns:
        if col == "Experiment":
            continue
        # Expecting columns like "berlin_PSNR"
        parts = col.split("_")
        scene = parts[0]
        metric = parts[-1]
        if scene in scene_values:
            if metric.upper() == "PSNR":
                row[col] = scene_values[scene][0]
            elif metric.upper() == "SSIM":
                row[col] = scene_values[scene][1]
            elif metric.upper() == "LPIPS":
                row[col] = scene_values[scene][2]
            else:
                row[col] = np.nan
        else:
            row[col] = np.nan
    baseline_rows.append(row)

# Concatenate the baseline rows with final_results.
final_results = pd.concat([final_results, pd.DataFrame(baseline_rows)], ignore_index=True)

overall_best = {}
best_experiment = {}
for col in final_results.columns:
    if col == "Experiment":
        continue
    metric = col.split("_")[-1].upper()
    if metric in ["PSNR", "SSIM"]:
        overall_best[col] = final_results[col].max()
        best_experiment[col] = final_results.loc[~final_results["Experiment"].isin(baseline_dict.keys()), col].max()
    elif metric == "LPIPS":
        overall_best[col] = final_results[col].min()
        best_experiment[col] = final_results.loc[~final_results["Experiment"].isin(baseline_dict.keys()), col].min()

def format_cell(value, col, row_type):
    try:
        num_val = float(value)
    except:
        return ""
    s = f"{num_val:.3f}"
    # Underline & bold if this is an experiment row and equals the best among experiments.
    if row_type == "experiment" and np.isclose(num_val, best_experiment[col], atol=1e-6):
        s = f"\\underline{{\\textbf{{{num_val:.3f}}}}}"
    # Highlight if equals the overall best.
    if np.isclose(num_val, overall_best[col], atol=1e-6):
        s = f"\\cellcolor{{red}}{s}"
    return s

formatted_final = final_results.copy()
for idx, row in final_results.iterrows():
    row_type = "baseline" if row["Experiment"] in baseline_dict.keys() else "experiment"
    for col in final_results.columns:
        if col == "Experiment":
            continue
        formatted_final.at[idx, col] = format_cell(row[col], col, row_type)

best_is_max = {"psnr": True, "ssim": True, "lpips": False}
new_columns = [("Experiment", "")]
for col in formatted_final.columns:
    if col == "Experiment":
        continue
    parts = col.split("_")
    scene, metric = parts[0], parts[-1]
    arrow = " $\\uparrow$" if best_is_max.get(metric.lower(), False) else " $\\downarrow$"
    new_columns.append((scene.capitalize(), metric.upper() + arrow))
multi_index = pd.MultiIndex.from_tuples(new_columns)
formatted_final.columns = multi_index

all_cols = list(formatted_final.columns)
other_cols = all_cols[1:]  # all except "Experiment"
n_other = len(other_cols)
split_idx = math.ceil(n_other / 2)
cols_part1 = [all_cols[0]] + other_cols[:split_idx]
cols_part2 = [all_cols[0]] + other_cols[split_idx:]
df_part1 = formatted_final[cols_part1]
df_part2 = formatted_final[cols_part2]

latex_table_part1 = df_part1.to_latex(escape=False, index=False, multicolumn=True, multicolumn_format='c')
latex_table_part2 = df_part2.to_latex(escape=False, index=False, multicolumn=True, multicolumn_format='c')

print("LaTeX Table Part 1:")
print(latex_table_part1)
print("\nLaTeX Table Part 2:")
print(latex_table_part2)

LaTeX Table Part 1:
\begin{tabular}{lllllll}
\toprule
Experiment & \multicolumn{3}{c}{Berlin} & \multicolumn{3}{c}{Nyc} \\
 & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ \\
\midrule
Global NeRF (1000its) + SfM & 26.251 & 0.886 & 0.223 & 26.450 & 0.827 & 0.334 \\
Local NeRFs+ SfM & \underline{\textbf{26.354}} & 0.887 & 0.221 & 26.557 & 0.828 & 0.332 \\
SfM & 26.204 & \underline{\textbf{0.888}} & \cellcolor{red}\underline{\textbf{0.217}} & \underline{\textbf{26.711}} & \underline{\textbf{0.830}} & \cellcolor{red}\underline{\textbf{0.327}} \\
EVER & \cellcolor{red}27.240 & \cellcolor{red}0.900 & 0.371 & \cellcolor{red}27.930 & \cellcolor{red}0.863 & 0.337 \\
GSplat-7k & nan & nan & nan & nan & nan & nan \\
3DGS MCMC & nan & nan & nan & nan & nan & nan \\
\bottomrule
\end{tabular}


LaTeX Table Part 2:
\begin{tabular}{llllll}
\toprule
Experiment & \multicolumn{2}{c}{London} & \multicolumn{3}{c}{Alameda} \\
 & PSNR $\uparro

/tmp/ipykernel_225123/2143060490.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '26.251' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  formatted_final.at[idx, col] = format_cell(row[col], col, row_type)
/tmp/ipykernel_225123/2143060490.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.886' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  formatted_final.at[idx, col] = format_cell(row[col], col, row_type)
/tmp/ipykernel_225123/2143060490.py:240: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.223' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  formatted_final.at[idx, col] = format_cell(row[col], col, r

In [140]:
def compute_scene_time_to_reach(scene, baseline_experiment="SfM"):
    data_by_metric = {}
    name_map = {}
    training_times = {}

    for metric in metrics_to_process:
        log_dir = f"../tensorboard_results/{scene}{alt}{local_text}/{metric}"
        dfs = []
        for root, dirs, files in os.walk(log_dir):
            for file in files:
                if file.endswith(".csv"):
                    # For berlin, skip files that contain "100k"
                    if "processed_splatfacto_sparse_pc_30000_its" not in file:
                        if scene != "stump":
                            if scene == "berlin" and "100k" in file:
                                continue
                            #elif scene == "bicycle" and (("1000" not in file) or "sparse" in file):
                            #    continue
                            #elif ("sparse_pc" not in file) and ((("150000" not in file) and ("150k" not in file))):
                            #    continue
                            #elif ("200" in file) or ("500its" in file) or ("100its" in file) or ("10000its" in file) or ("add" in file):
                            #    continue
                        elif "nerf_10000iters_init_30000_its" in file or "nerf_1000iters_init_30000_its" in file:
                            continue
                    full_path = os.path.join(root, file)
                    df = pd.read_csv(full_path)

                    experiment_name = os.path.splitext(file)[0]
                    experiment_name = experiment_name.split("splatfacto_")[-1]
                    experiment_name = (experiment_name.replace("1mil", "1000000")
                                                   .replace("1m", "1000000")
                                                   .replace("1k", "1000")
                                                   .replace("0k", "0000")
                                                   .replace("add_", ""))
                    df['experiment'] = experiment_name
                    dfs.append(df)

                    # Build the mapping and record training time if not already set.
                    if experiment_name not in name_map:
                        if ("merged_sfm" in experiment_name) or ("sfm_global" in experiment_name):
                            pattern = r'(\d+)its_(\d+)pts_(\d+)_its'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                points = int(match.group(2))
                                display_name = f"Global NeRF ({nerf_iterations}its) + SfM"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                pattern = r'(\d+)its_(\d+)_'
                                match = re.search(pattern, experiment_name)
                                if match:
                                    nerf_iterations = int(match.group(1))
                                    display_name = f"Global NeRF ({nerf_iterations}its) + SfM"
                                    name_map[experiment_name] = display_name
                                    training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                                else:
                                    name_map[experiment_name] = experiment_name
                        elif "sparse_pc" in experiment_name:
                            name_map[experiment_name] = "SfM"
                        elif (("local_merged" in experiment_name) or ("merged_local" in experiment_name)) and ("1000" in experiment_name):
                            name_map[experiment_name] = "Local NeRFs+ SfM"
                            training_times[name_map[experiment_name]] = local_training_times_by_scene.get(scene.split("_")[0], {}).get("1000_merged", 0.0)
                        elif "local_nerf" in experiment_name:
                            pattern = r'(\d+)its_(\d+)'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                points = int(match.group(2))
                                display_name = f"Local NeRFs ({nerf_iterations}its, {points} pts) + SfM"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = local_training_times_by_scene.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                name_map[experiment_name] = experiment_name
                        elif "global_nerf" in experiment_name:
                            pattern = r'(\d+)its_(\d+)_'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                points = int(match.group(2))
                                display_name = f"Global NeRF ({nerf_iterations}its, {points} pts)"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                name_map[experiment_name] = experiment_name
                        elif "nerf" in experiment_name:
                            pattern = r'(\d+)_iters'
                            match = re.search(pattern, experiment_name)
                            if match:
                                nerf_iterations = int(match.group(1))
                                display_name = f"Global NeRF ({nerf_iterations}its) only"
                                name_map[experiment_name] = display_name
                                training_times[display_name] = global_training_times.get(scene.split("_")[0], {}).get(nerf_iterations, 0.0)
                            else:
                                name_map[experiment_name] = experiment_name
                        else:
                            print("DROPPING", experiment_name)
                            continue
                            name_map[experiment_name] = experiment_name
        if dfs:
            data_by_metric[metric] = pd.concat(dfs, ignore_index=True)

    def compute_time_to_reach(df, experiments, baseline_experiment, best_is_max, by="time"):
        df = df[df['experiment'].isin(experiments.keys())].copy()
        df['experiment_display'] = df['experiment'].map(experiments)
        if "Time" not in df.columns and "Wall time" in df.columns:
            df = df.rename(columns={"Wall time": "Time"})
        x = "Time" if by=="time" else "Step"
        # Get the baseline data for the baseline_experiment.
        baseline_df = df[df['experiment_display'].str.lower() == baseline_experiment.lower()]
        if baseline_df.empty:
            threshold = None
            baseline_duration = float('inf')
        else:
            if best_is_max:
                threshold = baseline_df['Value'].max()
            else:
                threshold = baseline_df['Value'].min()
            baseline_duration = baseline_df[x].max() - baseline_df[x].min()
        time_dict = {}
        for exp, group in df.groupby('experiment_display'):
            if by=="time":
                startup = training_times.get(exp, 0.0)
                group = group.copy()
                group[x] = group[x] - group[x].min() + startup
            # Find the first time at which the experiment reaches the threshold.
            if threshold is None:
                time_dict[exp] = None
                continue
            if best_is_max:
                reached = group[group['Value'] >= threshold]
            else:
                reached = group[group['Value'] <= threshold]
            if reached.empty:
                time_dict[exp] = None
            else:
                time_dict[exp] = reached[x].iloc[0]
        return time_dict

    time_to_reach = {}
    for metric in metrics_to_process:
        if metric not in data_by_metric:
            continue
        times = compute_time_to_reach(data_by_metric[metric], name_map, baseline_experiment, best_is_max[metric], by="time")
        time_to_reach[metric] = times

    scene_time_results = {}
    for metric in metrics_to_process:
        if metric not in time_to_reach:
            continue
        col_name = f"Time {metric.upper()}"
        for exp, t in time_to_reach[metric].items():
            if exp not in scene_time_results:
                scene_time_results[exp] = {}
            scene_time_results[exp][col_name] = t
    results_df = pd.DataFrame.from_dict(scene_time_results, orient='index').reset_index().rename(columns={"index": "Experiment"})
    return results_df

results_time_per_scene = {}
for scene in scenes:
    df_scene_time = compute_scene_time_to_reach(scene, baseline_experiment="SfM")
    df_scene_time = df_scene_time.set_index("Experiment")
    rename_dict = {}
    for col in df_scene_time.columns:
        metric = col.split(" ")[1]
        rename_dict[col] = f"{scene}_{metric}"
    df_scene_time = df_scene_time.rename(columns=rename_dict)
    results_time_per_scene[scene] = df_scene_time

merged_time_results = None
for scene, df in results_time_per_scene.items():
    if merged_time_results is None:
        merged_time_results = df.copy()
    else:
        merged_time_results = merged_time_results.join(df, how="outer")
merged_time_results = merged_time_results.reset_index()

#print("Time-to-reach baseline table:")
#print(merged_time_results)

def compute_speedup_table(merged_df, baseline_experiment="SfM"):
    df = merged_df.copy()
    for col in df.columns:
        if col == "Experiment":
            continue
        baseline_val = df.loc[df["Experiment"] == baseline_experiment, col]
        if baseline_val.empty:
            continue
        baseline_val = baseline_val.iloc[0]
        def calc_speedup(x):
            try:
                x_val = float(x)
                if x_val == 0:
                    return "N/A"
                return baseline_val / x_val
            except:
                return "N/A"
        df[col] = df[col].apply(calc_speedup)
    return df

speedup_df = compute_speedup_table(merged_time_results, baseline_experiment="SfM")

def format_speedup(value):
    try:
        v = float(value)
        return f"{v:.2f}x"
    except:
        return "N/A"

for col in speedup_df.columns:
    if col == "Experiment":
        continue
    speedup_df[col] = speedup_df[col].apply(lambda x: float(x) if x != "N/A" else np.nan)

for col in speedup_df.columns:
    if col == "Experiment":
        continue
    max_val = speedup_df[col].max(skipna=True)
    def fmt_cell(x):
        if pd.isna(x):
            return "N/A"
        s = f"{x:.2f}x"
        if np.isclose(x, max_val, atol=1e-6):
            # Highlight best speedup.
            return f"\\cellcolor{{red}}{{{s}}}"
        return s
    speedup_df[col] = speedup_df[col].apply(fmt_cell)

new_columns = [("Experiment", "")]
for col in speedup_df.columns:
    if col == "Experiment":
        continue
    parts = col.split("_")
    scene, metric = parts[0], parts[-1]
    arrow = r" $\uparrow$" if best_is_max.get(metric.lower(), False) else r" $\downarrow$"
    new_columns.append((scene.capitalize(), metric.upper() + arrow))
speedup_df.columns = pd.MultiIndex.from_tuples(new_columns)

all_cols = list(speedup_df.columns)
other_cols = all_cols[1:]
split_idx = math.ceil(len(other_cols)/2)
cols_part1 = [all_cols[0]] + other_cols[:split_idx]
cols_part2 = [all_cols[0]] + other_cols[split_idx:]
df_speedup_part1 = speedup_df[cols_part1]
df_speedup_part2 = speedup_df[cols_part2]

latex_speedup_part1 = df_speedup_part1.to_latex(escape=False, index=False, multicolumn=True, multicolumn_format='c')
latex_speedup_part2 = df_speedup_part2.to_latex(escape=False, index=False, multicolumn=True, multicolumn_format='c')

print("\nLaTeX Speedup Table Part 1:")
print(latex_speedup_part1)
print("\nLaTeX Speedup Table Part 2:")
print(latex_speedup_part2)


LaTeX Speedup Table Part 1:
\begin{tabular}{lllllll}
\toprule
Experiment & \multicolumn{3}{c}{Berlin} & \multicolumn{3}{c}{Nyc} \\
 & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ \\
\midrule
Global NeRF (1000its) + SfM & 1.00x & 0.92x & 0.91x & N/A & 0.87x & 0.90x \\
Local NeRFs+ SfM & \cellcolor{red}{1.02x} & 0.96x & 0.94x & 0.89x & 0.94x & 0.93x \\
SfM & 1.00x & \cellcolor{red}{1.00x} & \cellcolor{red}{1.00x} & \cellcolor{red}{1.00x} & \cellcolor{red}{1.00x} & \cellcolor{red}{1.00x} \\
\bottomrule
\end{tabular}


LaTeX Speedup Table Part 2:
\begin{tabular}{llllll}
\toprule
Experiment & \multicolumn{2}{c}{London} & \multicolumn{3}{c}{Alameda} \\
 & PSNR $\uparrow$ & LPIPS $\downarrow$ & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ \\
\midrule
Global NeRF (1000its) + SfM & 0.99x & \cellcolor{red}{1.07x} & 1.05x & 1.04x & 1.02x \\
Local NeRFs+ SfM & \cellcolor{red}{1.01x} & 1.00x & \cellcolor{red}{1.07x} & \ce